# <center>***Posterior Probability*** <br>
<center>(coding from scratch)


In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Preparing the Dataset for classification

In [ ]:
data = pd.read_csv('../input/churn-modelling/Churn_Modelling.csv')

In [ ]:
print(data.head)

### Setting the required columns

In [ ]:
req_col = ['CreditScore','Geography','Gender','Age','NumOfProducts','Exited']
data = data[req_col]

### Checking the first 5 rows of the DataSet

In [ ]:
data.head

### We use this method to gather information about the DataSet

In [ ]:
data.info()

In [ ]:
#assign an arbitrary value according to the size of the dataset to get appropriate sections of the dataset
val = 5
prob = pd.DataFrame(columns = ['Attr','Value','C1','C2','P1','P2'])

### Setting the target
We choose Exited as the target since we are classifying the data into Exited or Did Not Exit the Bank

In [ ]:
target = 'Exited'

### Splitting the dataset into Exited and Did Not Exit

In [ ]:
#Exited
ex = data[data[target]==1]
#Did Not Exit
nex = data[data[target]==0]

### Checking the shape of both the dataframes

In [ ]:
print(nex.shape,ex.shape)

### Choose your input. Uncomment the desired input and Comment the desired one

In [ ]:
# For input as Did Not Exit
#input_val = nex.iloc[7005,:]
#print(input_val)


## For input as Exited
input_val = ex.iloc[2005,:]
print(input_val)

In [ ]:
nex = nex.iloc[0:2040,:]

In [ ]:
print(nex.shape,ex.shape)
nex_sh = nex.shape[0]/(ex.shape[0]+nex.shape[0])
ex_sh = ex.shape[0]/(ex.shape[0]+nex.shape[0])
print("P(C1) = {} \n P(C2) = {}".format(ex_sh, nex_sh))

In [ ]:
C1 = []
C2 = []

# Step 2: Calculating P(X/Ci) for classes

In [ ]:
for x in req_col:
    
    if (data[x].dtype == np.float64 or data[x].dtype == np.int64):
        if(len(pd.unique(data[x]))>(val)):
            m = data[x].value_counts(bins=3).reset_index().rename(columns={'index':'bin'})
            C1 = []
            C2 = []
            P1 = []
            P2 = []
            
            
            for check in m['bin']:
               
                
                t_c1 = int(ex[(check.left <= ex[x]) & (ex[x] <= check.right)].shape[0])
                t_p1 = int(ex[(check.left <= ex[x]) & (ex[x] <= check.right)].shape[0])/ex.shape[0]
                t_c2 = int(nex[(check.left <= nex[x]) & (nex[x] <= check.right)].shape[0])
                t_p2 = int(nex[(check.left <= nex[x]) & (nex[x] <= check.right)].shape[0])/nex.shape[0]
    
                C1.append(t_c1)
                C2.append(t_c2)
                P1.append(t_p1)
                P2.append(t_p2)
            #print(C1,C2,P1,P2)

            inc = pd.DataFrame({"Attr" : [x,x,x], "Value" : m['bin'], "C1" : C1, "C2" : C2, "P1" : P1, "P2" : P2})
            #print(inc)
            prob = prob.append(inc)
        else:
            C1 = []
            C2 = []
            P1 = []
            P2 = []
            uni = data[x].unique()
            for check in uni:
                if (check in ex[x].value_counts()):
                    C1.append(ex[x].value_counts()[check])
                    #print(C1)
                    P1.append(ex[x].value_counts()[check]/ex.shape[0])
                    
                else:
                    C1.append(0)
                    P1.append(0)
                    
                if (check in nex[x].value_counts()):
                    C2.append(nex[x].value_counts()[check])
                    #print(C2)
                    P2.append(nex[x].value_counts()[check]/nex.shape[0])
                    
                else:
                    C2.append(0)
                    P2.append(0)
                    
                
                    
            inc = pd.DataFrame({"Attr" : [x]*len(uni), "Value" : uni, "C1" : C1, "C2" : C2, "P1" : P1, "P2" : P2})
            prob = prob.append(inc)
                
    else:
        C1 = []
        C2 = []
        P1 = []
        P2 = []
        uni = data[x].unique()
        for check in uni:
            C1.append(ex[x].value_counts()[check])
            #print(C1)
            C2.append(nex[x].value_counts()[check])
            #print(C2)
            P1.append(ex[x].value_counts()[check]/ex.shape[0])
            P2.append(nex[x].value_counts()[check]/nex.shape[0])    
        inc = pd.DataFrame({"Attr" : [x]*len(uni), "Value" : uni, "C1" : C1, "C2" : C2, "P1" : P1, "P2" : P2})
        prob = prob.append(inc)
        

In [ ]:
prob = prob.reset_index().drop(['index'],axis = 1)

### Checking and Editing the Input Values to remove the Target Column

In [ ]:
print("\nInput Value:")
print(input_val)
print("\nExpected Value:")
input_val = input_val.drop(['Exited'])
print("\nRevised DataFrame")
print(input_val)

### Calculating the value of P(Ci/X)

In [ ]:
fin_P1 = 1
fin_P2 = 1
for inin in input_val.index:
    answer = prob[(prob['Attr'] == inin)]
    #print(answer)
    #print(inin)
    
    for val in answer['Value']:
        #print(type(val))
        if(isinstance(val, int)):
            if(input_val[inin] == val):
                ind = answer.index[answer.Value == val]
                fin_P2*=float(answer['P2'].loc[ind])
                #print(float(answer['P2'].loc[ind]))
                fin_P1*=float(answer['P1'].loc[ind])
                #print(float(answer['P1'].loc[ind]))
            
        else:
            if(input_val[inin] in val):
                ind = answer.index[answer.Value == val]
                fin_P2*=float(answer['P2'].loc[ind])
                #print(float(answer['P2'].loc[ind]))
                fin_P1*=float(answer['P1'].loc[ind])
                #print(float(answer['P1'].loc[ind]))
print("P(X/C1) = {}".format(fin_P1))         
print("P(X/C2) = {}".format(fin_P2))        

# Step 3: Calculating Posterior Probability

In [ ]:
print("P(C1/X) = {} * {} = {}".format(fin_P1, ex_sh,fin_P1*ex_sh ))
print("P(C2/X) = {} * {} = {}".format(fin_P2, nex_sh,fin_P2*nex_sh))

# Conclusion:

In [ ]:
if(fin_P1 * ex_sh > fin_P2 * nex_sh):
    print("Conclusion: Since P(X) is constant and common for both the classes and the maximum posterior is P(C1/X) = {}. Hence tuple/unknown sample gets classified as Exited".format(fin_P1*ex_sh))
else:
    print("Conclusion: Since P(X) is constant and common for both the classes and the maximum posterior is P(C2/X) = {}. Hence tuple/unknown sample gets classified as Did Not Exit".format(fin_P2*nex_sh))